In [1]:
import my_utils as mu
import pandas as pd
import json
import requests
import time
from tqdm import tqdm

In [2]:
def checkApiResult(url: str, riot_api_key: str):
    """
    `request.get()`의 결과에 따라서 API Key를 교체하거나 에러 메시지를 출력하고
    에러인 경우 `False`를 리턴하여 `if`가 걸린 부분의 `continue` 트리거를 작동시키는 `autoInsert()` 전용 내부 함수
    """
    while True:
        try:
            result = requests.get(url+riot_api_key)

            # 리턴이 정상인 경우 json 형태로 리턴
            if result.status_code == 200:
                return result.json()
            
            # 리미트 초과인 경우 10초 휴식 후 재시도
            elif result.status_code == 429:
                mu.apiSleep(10, True)
                continue

            # 만료 되었거나 불량인 API key 사용시 API key 관련 에러문 출력
            elif result.status_code == 403:
                raise mu.InvaildApiKey(result)
            
            # 리턴의 상태 코드가 200, 429, 403이 아닌 경우 예외로 간주하고 API 관련 에러문 출력 출력
            else:
                raise mu.BadApiResult(result)
                
        except requests.exceptions.ConnectionError as e:
            time.sleep(5)

        # 위에서 발생된 모든 예외를 캐치하여 API 관련 에러 카운트를 1 올리고 False를 리턴(continue 트리거 작동)
        except Exception as e:
            print(f"{type(e).__name__}:\n{e}")
            return False

In [3]:
def reloadInsert(summoner_name: str, riot_api_key: str, debug: bool=False):

    # Summoner 테이블의 컬럼 리스트
    summoner_table_cols = [
        'summoner_id',
        'summoner_puuid',
        'api_key',
        'summoner_name',
        'summoner_level',
        'summoner_profile',
        'summoner_tier',
        'summoner_wins',
        'summoner_losses',
        'summoner_veteran',
        'summoner_inactive',
        'summoner_freshblood',
        'summoner_hotstreak']

    # Summoner 테이블의 일반 정보 키 리스트
    normal_data_keys = ['id', 'puuid', 'name', 'summonerLevel', 'profileIconId']

    # Summoner 테이블의 랭크 관련 정보 키 리스트
    rank_data_keys = ['tier', 'wins', 'losses', 'veteran', 'inactive', 'freshBlood', 'hotStreak']

    # 소환사 이름으로 최근 20게임의 매치 id를 획득
    if (puuid := checkApiResult(f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}?api_key=", riot_api_key) is False):
        return puuid
    if (match_id_list := checkApiResult(f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&type=ranked&api_key=", riot_api_key) is False):
        return match_id_list

    for match_id in match_id_list:

        # RawData 테이블에 현재 match_id가 이미 존재한다면 (# 리스트에서 제거 후) continue
        if 0 != mu.oracle_totalExecute(f"SELECT COUNT(game_id) FROM RAWDATA WHERE game_id = '{match_id}'", use_pandas=False, debug_print=False)[0][0]:
            # match_id_list.remove(match_id)
            continue
        
        # 현재 matchId의 matches와 timeline을 획득
        match_raw = {
            'matches': checkApiResult(f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key=", riot_api_key),
            'timeline': checkApiResult(f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key=", riot_api_key)}
        if True in [v is False for v in match_raw.values()]:
            return match_raw
            # logging.error({"time": time.strftime('%Y-%m-%d %H:%M:%S'), "errorType": "apiMatchData", "apiKey": riot_api_key, "dataType": "matchId", "data": match_id})
            # continue

        # RawData 테이블 형태로 가공하여 저장
        filterd_match_raw = mu.RawdataFirstFilter(pd.DataFrame([match_raw]), riot_api_key)

        # 가공된 데이터가 비정상일 경우 continue
        if filterd_match_raw.__class__ is not list:
            return filterd_match_raw
            # logging.error({"time": time.strftime('%Y-%m-%d %H:%M:%S'), "errorType": "missingGameData", "apiKey": riot_api_key, "dataType": "dict",
            #                 "data": {"error": str(f"{filterd_match_raw.__class__.__name__}: {filterd_match_raw}"), "matchId": match_id}})
            # continue

        # 현재 게임에 참가중인 모든 플레이어의 puuid 추출후 중복을 방지하기 위해 현재 유저만 제거
        match_puuid_list = match_raw['matches']['metadata']['participants']
        match_puuid_list.remove(puuid)

        # 현재 게임의 소환사 데이터 저장용 리스트
        match_part_summoner_data_list = []

        # 현재 게임에서 9명의 puuid로 Summoner 테이블에 필요한 정보를 추출 및 삽입
        print(f"\t\t'{match_id}': 게임 참여자의 유저 정보 추출 및 삽입......")
        for part_puuid in match_puuid_list:

            # puuid로 소환사 정보 획득
            if (part_summoner_detail := checkApiResult(f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{part_puuid}?api_key=", riot_api_key)) is False:
                return part_summoner_detail
                # logging.error({"time": time.strftime('%Y-%m-%d %H:%M:%S'), "errorType": "apiSummonerData", "apiKey": riot_api_key, "dataType": "puuId", "data": part_puuid})
                # continue
            part_summoner_normal_data = [part_summoner_detail[i] for i in normal_data_keys]
            part_summoner_normal_data.insert(2, riot_api_key)
            
            # summonerId로 랭크 관련 정보 획득
            if (part_rank_detail := checkApiResult(f"https://kr.api.riotgames.com/lol/league/v4/entries/by-summoner/{part_summoner_normal_data[0]}?api_key=", riot_api_key)) is False:
                return part_rank_detail
                # logging.error({"time": time.strftime('%Y-%m-%d %H:%M:%S'), "errorType": "apiSummonerRankData", "apiKey": riot_api_key, "dataType": "summonerId", "data": part_summoner_normal_data[0]})
                # continue
            
            # 이 유저가 솔랭에서 언랭 혹은 배치이면서 다른 랭크 데이터가 전혀 없다면 디폴트로 삽입
            if part_rank_detail == []:
                part_summoner_rank_data = ['DEFAULT' for i in rank_data_keys]

            # 이 유저가 솔랭에서 언랭 혹은 배치이면서 다른 랭크 데이터가 있다면 필요한 랭크 데이터가 없기 때문에 디폴트로 삽입
            elif not True in (is_solo_rank_list := [a['queueType'] == 'RANKED_SOLO_5x5' for a in part_rank_detail]):
                part_summoner_rank_data = ['DEFAULT' for i in rank_data_keys]

            # summonerId의 정상 여부 확인 후 솔랭 데이터만 찾기
            else:
                part_solo_rank_detail = part_rank_detail[is_solo_rank_list.index(True)]
                part_summoner_rank_data = [part_solo_rank_detail[i] for i in rank_data_keys]

            # 랭크 관련 정보의 bool 값을 0,1로 변환
            for idx in range(3, 7):
                if part_summoner_rank_data[idx]: part_summoner_rank_data[idx] = 1
                else: part_summoner_rank_data[idx] = 0

            # 9명의 정보를 현재 게임의 소환사 정보 리스트에 추가
            match_part_summoner_data_list.append(part_summoner_normal_data + part_summoner_rank_data)
        
        # 현재 게임의 게임 데이터와 유저 데이터 전부 확인 후 삽입
        if not debug: mu.insertDataFrameIntoTable(pd.DataFrame(filterd_match_raw), 'RAWDATA', debug_print=False)

        if not debug: mu.insertDataFrameIntoTable(pd.DataFrame(match_part_summoner_data_list, columns=summoner_table_cols), 'SUMMONER', debug_print=False)

In [14]:
mu.oracle_totalExecute(f"SELECT COUNT(game_id) FROM RAWDATA WHERE game_id = 'KR_6502628728'", use_pandas=False, debug_print=False)[0][0]

10

In [5]:
reloadInsert('기본적으로못함', '이세인1')

In [15]:
sql = f"""
    SELECT *
    FROM (
        SELECT GAME_ID
        FROM RAWDATA
        WHERE PARTICIPANT_NAME = '기본적으로못함'
        ORDER BY GAME_ID DESC)
    WHERE ROWNUM <= 20
"""
game_ids = mu.oracle_totalExecute(sql)

oracle open!
oracle close!


In [17]:
game_ids

,GAME_ID
0,KR_6484431751
1,KR_6484396115
